In [1]:
import numpy as np
import shorttext

//anaconda/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [6]:
# loading the whole ESV Bible
import biblebooks.bibledocs_iterator as bit
dbconn = bit.get_sqlite3_dbconn("ESVBible2018.sqlite")
biblecorpus = bit.generate_classdict_chapters(dbconn)

In [12]:
from biblebooks import BibleAbbrDict

In [5]:
# loading FastText model
wvmodel = shorttext.utils.load_fasttext_model('/Users/hok/Data/FastText/wiki.simple/wiki.simple.bin')

In [4]:
# define neural network

In [7]:
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, LSTM, Activation, Input
from keras.models import Sequential, Model
from keras.regularizers import l2

In [8]:
input_x = Input(shape=(None, wvmodel.vector_size))
output_y = LSTM(1024, return_sequences=True)(input_x)
output_y = Conv1D(1200, 120)(output_y)
output_y = MaxPooling1D(100)(output_y)
output_y = Dense(2, activation='sigmoid')(output_y)

kmodel = Model(inputs=input_x, outputs=output_y)

In [9]:
kmodel.compile(loss='categorical_crossentropy', optimizer='adam')

In [10]:
wvmodel.vector_size

300

In [18]:
newdict = {}
newdict['ot'] = [sent for otbookabbr in BibleAbbrDict.otbookdict 
                 for chap in range(1, BibleAbbrDict.numchaps[otbookabbr]+1) 
                 for sent in biblecorpus[otbookabbr+'_'+str(chap)]]
newdict['nt'] = [sent for otbookabbr in BibleAbbrDict.ntbookdict 
                 for chap in range(1, BibleAbbrDict.numchaps[otbookabbr]+1) 
                 for sent in biblecorpus[otbookabbr+'_'+str(chap)]]

In [22]:
classifier = shorttext.classifiers.VarNNEmbeddedVecClassifier(wvmodel)

In [ ]:
classifier.train(newdict, kmodel)